In [2]:
from src.layers.Models import TinyDiffusion 
from src.layers.seq2seq import Seq2Seq
from torch.utils.data import DataLoader
from src.data.BaseData import DataSetTinyDiffusion
from src.utils.manage_device import select_device

/Data/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
sequence_model = Seq2Seq(
    depth=2 , 
    size = 256 , 
    hidden_dim=256 ,
    output_dim=2 , 
    pos_emb="sinusoidal" ,
    input_size=2 , 
    device="cuda"
)

torch.Size([1, 1, 256])


In [17]:
data_set = DataSetTinyDiffusion(name_dataset=("dino" , "") , **{
'number_data_points': 10000, 'joined': 0, 'device' : 'cuda:0'
}
) 
data_loader = DataLoader(data_set, batch_size=200, shuffle=True)

In [18]:
device = "cuda"

In [19]:
cfg ={'device': 'cuda', 
      'model_type': {'model_name': 'Residual_only'}, 
      'model': {'depth': 3, 'size': 250, 'hidden_dim': 750, 'output_dim': 2, 'pos_emb': 'sinusoidal', 'input_size': 2}, 
      'trainer': {'num_epoch': 50, 'batch_size': 64, 'gradient_clip_threshhold': 1, 'eval_frequency': 10, 'sample_size': 2000}, 
      'dataset': {'number_data_points': 10000, 'joined': 0, 'name_dataset': [['dino', ''], ['mnist', 'density_map_0.pt']]}, 
      'optimizer': {'lr': 0.001, 'weight_decay': 0}, 
      'Diffusion': {'num_timesteps': 250, 'beta_start': 0.0001, 'beta_end': 0.02, 'beta_schedule': 'quadratic'}}
 

In [20]:
optimizer_config = {**cfg["optimizer"]}
diff_config = {**cfg["Diffusion"], "device": device}

In [21]:
import torch
from src.layers.model import Noise_Scheduler
optimizer = torch.optim.Adam(sequence_model.parameters(), **optimizer_config)
diffusion = Noise_Scheduler(**diff_config)


In [22]:
from src.utils.training import Trainer


In [23]:
trainer = Trainer(
    model = sequence_model , 
    noise_scheduler= diffusion ,
    optimizer = optimizer ,
    criterion= torch.nn.MSELoss() ,
    device= device , 
)

In [ ]:
trainer.train_seq2seq(num_epochs=100 , batch_size=200 , gradient_clipthres= 1 , train_loader=data_loader )